<h1 align=center><font size = 6>Población de las áreas sanitarias de Galicia</font></h1>

Este cuaderno calcula el número de habitantes de cada área sanitaria de Galicia.

Parte de un fichero del <a href="https://github.com/lipido/galicia-covid19">usuario lipido</a> con la correspondencia entre los ayuntamientos y las áreas sanitarias, y tras obtener directamente de la Wikipedia el número de habitantes de cada ayuntamiento, fusiona ambos datasets consiguiendo así la población por área. 

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import json
import requests
import lxml
import sys

Para tener los datos de población de cada ayuntamiento, toma los datos directamente de la Wikipedia

In [2]:
#Lee los datos de la wikipedia
url = 'https://gl.wikipedia.org/wiki/Concellos_de_Galicia'
headers = {'User-Agent': 
           'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

readWiki = requests.get(url, headers=headers).text
soup = BeautifulSoup(readWiki,'lxml')
table = soup.find_all('table')
tab = str(table[1]) #Se ha comprobado que la tabla que nos interesa es la index=1
list_data = pd.read_html(tab)
df_wiki = list_data[0]

#Hacemos algunos ajustes de formato en el dataframe, teniendo en cuenta que la población figura con decimales
df_wiki.drop(['Superficie(km²)', 'Capital', 'Comarca', 'Provincia', 'Alcaldía(agosto de 2017)'], axis=1, inplace=True)
habitantes = df_wiki['Habitantes(2017)']
df_wiki.drop(['Habitantes(2017)'], axis=1, inplace=True)
hab_corregido = []
for h in habitantes:
    if h.is_integer():
        hab_corregido.append(h)
    else:
        hab_corregido.append(h*1000)
df_wiki['Habitantes'] = hab_corregido
df_wiki = df_wiki.astype({'Habitantes':int})

df_wiki.head()

,Concello,Habitantes
0,Abadín,2490
1,Abegondo,5467
2,Agolada,2431
3,Alfoz,1781
4,Allariz,6026


Dado que hasta la fecha los datos proporcionados están organizados en áreas sanitarias, leeremos el fichero que incluye la correspondencia entre ayuntamientos y áreas. Posteriormente fusionaremos este dataset con el de la población, obteniendo así la población de cada área.

In [5]:
#Lee el fichero de las áreas sanitarias y ayuntamientos de Galicia
df_ayun = pd.read_csv('areas.csv')
df_ayun.rename(columns={'municipio':'Concello','provincia':'Provincia','area_sanitaria':'Area sanitaria'}, inplace=True)

#Fusiona el dataframe de las áreas sanitarias y población
df_pop = pd.merge(how='inner', 
                  left=df_wiki[['Concello','Habitantes']], 
                  right=df_ayun, 
                  left_on='Concello', 
                  right_on='Concello')

#Agrupa los valores para tener los datos de población de las áreas sanitarias
df_areas = df_pop.groupby(['Area sanitaria']).sum()
df_areas.reset_index(inplace=True)

#Guarda como csv
df_areas.to_csv('poblacion.csv', index=False)

df_areas

,Area sanitaria,Habitantes
0,A Coruña,550024
1,Ferrol,192136
2,Lugo,333631
3,Ourense,311674
4,Pontevedra,307877
5,Santiago,448118
6,Vigo,564814
